### Задача поиск похожих по эмбеддингам
Скачиваем датасет

!wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

Или данные можно скачать тут https://github.com/IlyaGusev/gazeta

Пример работы с ним:


```
from corus import load_ods_gazeta
path = 'gazeta.csv.gz'
records = load_ods_gazeta(path)
next(records)

```


что надо сделать:
1. на основе word2vec/fasttext реализовать метод поиска ближайших статей (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
2. Проверить насколько хорошо работают подходы

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install natasha stop_words annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 24.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 43.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 78.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32896 sha256=422aa57e752cc47886c69541c71cda4fb7a5d662a99bc4241d0abd0cc09b303e
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552453 sha256=3aadf6162f8edf2616bb181122a5c6379b5612b5afcd5b730e6dc55c7c0bc748
  

In [3]:
import string
import pickle
import annoy
import numpy as np
import pandas as pd

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec, FastText

In [4]:
!wget https://www.dropbox.com/s/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz

--2023-10-18 20:55:58--  https://www.dropbox.com/s/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz [following]
--2023-10-18 20:55:59--  https://www.dropbox.com/s/raw/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc435e2908903fc487763dff2b19.dl.dropboxusercontent.com/cd/0/inline/CF2PpKXRA_gcDwVUMNi0pKwQdUVt5cvKEOQFfTVNXhfe_kiVqD-amF5OvP0-j9dBMCjpGTTujNd8w1VFFmm8a2V_NuwTxkvcHmMzXms3PNvNnNe-eQ2OQcIy2V5s7ije5JA/file# [following]
--2023-10-18 20:56:00--  https://uc435e2908903fc487763dff2b19.dl.dropboxusercontent.com/cd/0/inline/CF2PpKXRA_gcDwVUMNi0pKwQdUVt5cvKEOQFfTVNXhfe_kiVqD-amF5OvP0-j9dBMCjpGTTujNd8w1VFFmm8a2V_NuwTxkvcHmMz

In [5]:
!tar -xzvf gazeta_jsonl.tar.gz

gazeta_train.jsonl
gazeta_test.jsonl
gazeta_val.jsonl


Создадим датафрейм на основе всех доступных данных.

In [6]:
df_train = pd.read_json(path_or_buf='gazeta_train.jsonl', lines=True)
df_val = pd.read_json(path_or_buf='gazeta_val.jsonl', lines=True)
df_test = pd.read_json(path_or_buf='gazeta_test.jsonl', lines=True)

df = pd.concat([df_train, df_val, df_test], ignore_index=True)
df.head()

,url,text,title,summary,date
0,https://www.gazeta.ru/financial/2011/11/30/385...,«По итогам 2011 года чистый отток может состав...,Прогноз не успевает за оттоком,"В 2011 году из России уйдет $80 млрд, считают ...",2011-11-30 18:33:39
1,https://www.gazeta.ru/business/2013/01/24/4939...,Российское подразделение интернет-корпорации G...,Google закончил поиск,"Юлия Соловьева, экс-директор холдинга «Профмед...",2013-01-24 18:20:09
2,https://www.gazeta.ru/social/2018/02/06/116393...,Басманный районный суд Москвы вечером 6 феврал...,«Фигуранты дела могут давить на свидетелей»,Суд арестовал на два месяца четверых экс-чинов...,2018-02-06 21:21:14
3,https://www.gazeta.ru/business/2013/06/21/5388...,Как повлияло вступление в ВТО на конкурентносп...,«С последних традиционно «отжимают» больше»,Мнения предпринимателей по поводу вступления в...,2013-06-21 17:43:50
4,https://www.gazeta.ru/culture/2014/12/27/a_636...,К третьему сезону «Голос» на Первом канале ста...,Третий «Голос» за Градского,На Первом канале завершился третий сезон шоу «...,2014-12-27 01:10:01


В качестве основы для обучения моделей будем использовать столбец с summary статей, так как он отражает общую суть, но при этом значительно короче, а значит обучение будет быстрее.

In [7]:
text_lengths = df['text'].apply(len)
summary_lengths = df['summary'].apply(len)

print(f'Средняя длина текста статьи - {np.average(text_lengths):.0f} слов.')
print(f'Средняя длина summary - {np.average(summary_lengths):.0f} слов.')

Средняя длина текста статьи - 4519 слов.
Средняя длина summary - 312 слов.


In [8]:
morpher = MorphAnalyzer()
stopwords = set(get_stop_words("ru"))
punkt = set(string.punctuation)

def preprocess_txt(line, morpher=morpher, sw=stopwords, punkt=punkt):
    spls = "".join(i for i in line.strip() if i not in punkt).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

df['preprocessed_summary'] = df['summary'].apply(preprocess_txt)
df.head()

,url,text,title,summary,date,preprocessed_summary
0,https://www.gazeta.ru/financial/2011/11/30/385...,«По итогам 2011 года чистый отток может состав...,Прогноз не успевает за оттоком,"В 2011 году из России уйдет $80 млрд, считают ...",2011-11-30 18:33:39,"[2011, россия, уйти, 80, млрд, считать, минэко..."
1,https://www.gazeta.ru/business/2013/01/24/4939...,Российское подразделение интернет-корпорации G...,Google закончил поиск,"Юлия Соловьева, экс-директор холдинга «Профмед...",2013-01-24 18:20:09,"[юлия, соловьёв, эксдиректор, холдинг, «профме..."
2,https://www.gazeta.ru/social/2018/02/06/116393...,Басманный районный суд Москвы вечером 6 феврал...,«Фигуранты дела могут давить на свидетелей»,Суд арестовал на два месяца четверых экс-чинов...,2018-02-06 21:21:14,"[суд, арестовать, месяц, четверо, эксчиновник,..."
3,https://www.gazeta.ru/business/2013/06/21/5388...,Как повлияло вступление в ВТО на конкурентносп...,«С последних традиционно «отжимают» больше»,Мнения предпринимателей по поводу вступления в...,2013-06-21 17:43:50,"[мнение, предприниматель, повод, вступление, в..."
4,https://www.gazeta.ru/culture/2014/12/27/a_636...,К третьему сезону «Голос» на Первом канале ста...,Третий «Голос» за Градского,На Первом канале завершился третий сезон шоу «...,2014-12-27 01:10:01,"[канал, завершиться, сезон, шоу, «голос», побе..."


In [10]:
window_size = 5
vector_size = 300

modelW2V = Word2Vec(sentences=df['preprocessed_summary'],
                    vector_size=vector_size, window=window_size,
                    min_count=1)
modelFT = FastText(sentences=df['preprocessed_summary'],
                   vector_size=vector_size, window=window_size,
                   min_count=1)

Обучим модели и создадим функцию для получения ближайших статей с помощью построенных моделей.

In [11]:
w2v_index = annoy.AnnoyIndex(vector_size, 'angular')
ft_index = annoy.AnnoyIndex(vector_size, 'angular')

index_map = {}
counter = 0


for i in range(len(df['preprocessed_summary'])):
    n_w2v = 0
    n_ft = 0
    index_map[i] = df['text'][i]

    vector_w2v = np.zeros(vector_size)
    vector_ft = np.zeros(vector_size)
    for word in df['preprocessed_summary'][i]:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index.add_item(i, vector_w2v)
    ft_index.add_item(i, vector_ft)


w2v_index.build(10)
ft_index.build(10)

True

In [12]:
def get_response(question, index, model, index_map, nn=5):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, nn, )
    return [index_map[i] for i in answers]

In [13]:
TEXT = "интересные фильмы"

Результаты поиска ближайших статей с помощью Word2Vec

In [14]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['Из-за безудержной добычи полезных ископаемых мир к 1999 году необратимо изменился. Нефть стала стоить $2 тыс., вода стала дефицитом, а все вокруг подмяли под себя корпорации. Хороший и честный полицейский Артур Бэйли ( Алан Ричсон , один из трибутов во второй части «Голодных игр») во время очередной охоты на похитителей питьевой воды вместе с напарником оказывается свидетелем старта «Кровавой гонки» — машины, в которой едут с помощью специальных двигателей, потребляющих в качестве топлива человеческую кровь. Когда полицейского ловят, организатор гонки, называющий себя Джулианом Недоноском ( Колин Каннингем ), решает сделать Артура новым участником — и с помощью подручных включает его в одну команду с бесшабашной Грейс Дарженто (Кристина Очоа). Попытка Артура сбежать оказывается неудачной, высокие принципы — он не желает никого убивать ради победы — едва не стоят жизни ему и Грейс. И в итоге он решает помочь напарнице победить и получить приз в десять миллионов, чтобы иметь возможност

с помощью FastText

In [15]:
get_response(TEXT, ft_index, modelFT, index_map)

['Молодая девушка Рита легко находит общий язык со зверями, особенно ей полюбился медвежонок, которого нашли в лесу и передали в зоопарк. Но с началом войны мирная жизнь заканчивается – Маргарита ( Ольга Погодина ) оказывается в немецком плену, затем в Гамбурге, где подобравший ее артист Отто фон Штауб (Паоло Контичини) делает ее солисткой кабаре. Через четыре года, после Победы, она возвращается в Ригу и устраивается посудомойкой в цирковой буфет. Но любовь к зверям — а теперь и к сцене – никуда не делась. Сначала она становится цирковой гонщицей, ездит на мотоцикле внутри железного шара. Потом ее способность приручать диких животных замечают два опытных дрессировщика — Борис Эдер ( Николай Добрынин ) и Константин Константиновский ( Андрей Чернышов ). Они решают учить молодую девушку своей опасной профессии. Первый канал начинает показ биографического сериала «Маргарита Назарова» — рассказ о жизни знаменитой дрессировщицы тигров и легенды советского цирка (полностью фильм доступен на 